In [40]:
df_filial = spark.sql("select * from aula_hive.tb_filial")

In [3]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql import functions as f
import os
import re

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [5]:
df_categoria = spark.sql("select * from aula_hive.tb_categoria")
df_pedido = spark.sql("select * from aula_hive.tb_pedido")
df_item_pedido = spark.sql("select * from aula_hive.tb_item_pedido")
df_cliente = spark.sql("select * from aula_hive.tb_cliente")
df_produto = spark.sql("select * from aula_hive.tb_produto")
df_cidade = spark.sql("select * from aula_hive.tb_cidade")
df_estado = spark.sql("select * from aula_hive.tb_estado")
df_filial = spark.sql("select * from aula_hive.tb_filial")
df_parceiro = spark.sql("select * from aula_hive.tb_parceiro")
df_subcategoria = spark.sql("select * from aula_hive.tb_subcategoria")

In [6]:
df_filial.count()

1738

In [7]:
df_pedido.createOrReplaceTempView('vw_pedido')
df_item_pedido.createOrReplaceTempView('vw_item_pedido')
df_categoria.createOrReplaceTempView('vw_categoria')
df_cliente.createOrReplaceTempView('vw_cliente')
df_produto.createOrReplaceTempView('vw_produto')
df_cidade.createOrReplaceTempView('vw_cidade')
df_estado.createOrReplaceTempView('vw_estado')
df_filial.createOrReplaceTempView('vw_filial')
df_parceiro.createOrReplaceTempView('vw_parceiro')
df_subcategoria.createOrReplaceTempView('vw_subcategoria')

In [8]:
df_filial.printSchema()

root
 |-- id_filial: string (nullable = true)
 |-- ds_filial: string (nullable = true)
 |-- id_cidade: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [9]:
df_pedido.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- dt_pedido: string (nullable = true)
 |-- id_parceiro: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- id_filial: string (nullable = true)
 |-- vr_total_pago: string (nullable = true)
 |-- dt_foto: string (nullable = true)



In [10]:
sql = ''' 
SELECT *
  FROM vw_categoria 
'''
df_stage = spark.sql(sql).show(10)

+------------+---------------+-------------+-------+
|id_categoria|   ds_categoria|perc_parceiro|dt_foto|
+------------+---------------+-------------+-------+
|           1|Categoria - 001|          2.0| 202306|
|           2|Categoria - 002|          2.0| 202306|
|           3|Categoria - 003|          2.0| 202306|
|           4|Categoria - 004|          2.0| 202306|
|           5|Categoria - 005|          5.0| 202306|
|           6|Categoria - 006|          1.0| 202306|
|           7|Categoria - 007|          5.0| 202306|
|           8|Categoria - 008|          3.0| 202306|
|           9|Categoria - 009|          5.0| 202306|
|          10|Categoria - 010|          6.0| 202306|
+------------+---------------+-------------+-------+
only showing top 10 rows



In [11]:
query = ''' 
SELECT p.id_pedido
        ,p.dt_pedido
        ,p.id_parceiro
        ,p.id_cliente
        ,p.id_filial
        ,p.vr_total_pago
        ,it.id_produto
        ,it.quantidade
        ,it.vr_unitario
        ,pr.ds_produto
        ,pr.id_subcategoria
        ,f.ds_filial
        ,f.id_cidade
        ,c.ds_cidade
        ,c.id_estado
        ,e.ds_estado
        ,par.nm_parceiro
        ,cli.nm_cliente
        ,cli.flag_ouro
        ,s.ds_subcategoria
        ,s.id_categoria
        ,cat.ds_categoria
        ,cat.perc_parceiro
  FROM vw_pedido p 
  JOIN vw_item_pedido it 
    ON it.id_pedido = p.id_pedido
  JOIN vw_produto pr 
    ON pr.id_produto = it.id_produto
  JOIN vw_filial f 
    ON f.id_filial = p.id_filial
  JOIN vw_cidade c
    ON c.id_cidade = f.id_cidade
  JOIN vw_estado e
    ON e.id_estado = c.id_estado
  JOIN vw_parceiro par
    ON par.id_parceiro = p.id_parceiro
  JOIN vw_cliente cli
    ON cli.id_cliente = p.id_cliente
  JOIN vw_subcategoria s
    ON s.id_subcategoria = pr.id_subcategoria
  JOIN vw_categoria cat
    ON cat.id_categoria = s.id_categoria
  
'''



In [15]:
df_stage = spark.sql(query)

In [16]:
df_stage = (df_stage
            .withColumn('Ano', year(df_stage.dt_pedido))
            .withColumn('Mês', month(df_stage.dt_pedido))
            .withColumn('Dia', dayofmonth(df_stage.dt_pedido))
            .withColumn('Trimestre', quarter(df_stage.dt_pedido))
           )

In [58]:
df_stage.withColumn('Ano',year(df_stage.dt_pedido))

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [17]:
df_stage = (df_stage
            .withColumn('Ano', year(df_stage.dt_pedido))
            .withColumn('Mês', month(df_stage.dt_pedido))
            .withColumn('Dia', dayofmonth(df_stage.dt_pedido))
            .withColumn('Trimestre', quarter(df_stage.dt_pedido))
           )

df_stage.show(truncate=False)

+----------+------------------------+-----------+----------+---------+-------------+----------+----------+-----------+--------------------+---------------+---------------+---------+---------------+---------+---------+--------------------+---------------------------+---------+---------------------+------------+---------------+-------------+----+---+---+---------+
|id_pedido |dt_pedido               |id_parceiro|id_cliente|id_filial|vr_total_pago|id_produto|quantidade|vr_unitario|ds_produto          |id_subcategoria|ds_filial      |id_cidade|ds_cidade      |id_estado|ds_estado|nm_parceiro         |nm_cliente                 |flag_ouro|ds_subcategoria      |id_categoria|ds_categoria   |perc_parceiro|Ano |Mês|Dia|Trimestre|
+----------+------------------------+-----------+----------+---------+-------------+----------+----------+-----------+--------------------+---------------+---------------+---------+---------------+---------+---------+--------------------+---------------------------+----

In [18]:
df_stage.show()

+----------+--------------------+-----------+----------+---------+-------------+----------+----------+-----------+--------------------+---------------+---------------+---------+---------------+---------+---------+--------------------+--------------------+---------+--------------------+------------+---------------+-------------+----+---+---+---------+
| id_pedido|           dt_pedido|id_parceiro|id_cliente|id_filial|vr_total_pago|id_produto|quantidade|vr_unitario|          ds_produto|id_subcategoria|      ds_filial|id_cidade|      ds_cidade|id_estado|ds_estado|         nm_parceiro|          nm_cliente|flag_ouro|     ds_subcategoria|id_categoria|   ds_categoria|perc_parceiro| Ano|Mês|Dia|Trimestre|
+----------+--------------------+-----------+----------+---------+-------------+----------+----------+-----------+--------------------+---------------+---------------+---------+---------------+---------+---------+--------------------+--------------------+---------+--------------------+--------